In [1]:
import xarray as xr
import pandas as pd
import locale

# locale.setlocale(locale.LC_ALL,'en_US')
# locale.setlocale(locale.LC_ALL,'en_US')
dataset = xr.open_dataset('MiningProcess_Flotation_Plant_Database.h5', engine = 'h5netcdf')
df = dataset.to_pandas()


In [2]:
## Resampling
df = df.resample('1h').mean()

##
df.iloc[:-1, df.columns.get_loc('% Iron Feed')] = df['% Iron Feed'].iloc[1:].values
df.iloc[:-1, df.columns.get_loc('% Silica Feed')] = df['% Silica Feed'].iloc[1:].values
df.iloc[:-1, df.columns.get_loc('% Iron Concentrate')] = df['% Iron Concentrate'].iloc[1:].values
df.iloc[:-1, df.columns.get_loc('% Silica Concentrate')] = df['% Silica Concentrate'].iloc[1:].values

df = df.iloc[:-1]

## Split into input and output
X = df.iloc[:,:-2]
Y = df['% Silica Concentrate']

In [3]:
# Adding lagged variables
import numpy as np
for lag in range(1,6):
    lagged_variable = np.zeros(len(df))
    lagged_variable[lag:] = Y.iloc[:-lag]


    X.loc[:,f'% Silica Concentrate Lag {lag}'] = lagged_variable


X = X[5:]

/tmp/ipykernel_53561/1270065800.py:8: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  X.loc[:,f'% Silica Concentrate Lag {lag}'] = lagged_variable


In [4]:
# Split to calibration-validation set and test set
X_cal = X['20170615':'20170723']
Y_cal = Y['20170615':'20170723']
X_test = X['20170412':'20170506']
Y_test = Y['20170412':'20170506']

In [5]:
# Remove missing values in between
X_test.loc['20170421':'20170424'] = pd.NA
X_test.dropna(inplace=True)

Y_test.loc['20170421':'20170424'] = pd.NA
Y_test.dropna(inplace=True)

/tmp/ipykernel_53561/2442657920.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  X_test.dropna(inplace=True)


In [6]:
# Store data

with pd.HDFStore('MiningProcess_Flotation_Plant_Database_train_split.h5') as store:
    store.put('X_cal', X_cal)
    store.put('Y_cal', Y_cal)
    store.put('X_test', X_test)
    store.put('Y_test', Y_test)